<a href="https://colab.research.google.com/github/kthur/kthur/blob/master/%5BAI_Essential%5D_1028_3%EC%9D%BC%EC%B0%A8_%EC%8B%A4%EC%8A%B5_%EA%B0%80%EC%9D%B4%EB%93%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install JAEN torchinfo transformers -qU

In [ ]:
import JAEN

print(JAEN.__version__) # 버전확인

0.1.0


In [ ]:
# 필요 도구 import
import torch
import torch.nn as nn
import torch.optim as optim
from torchinfo import summary
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from JAEN.utils import train_and_evaluate, plot_training_results


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
# FashionMNIST 데이터 변환 (이미지를 텐서로 변환하고 [0, 1] 범위로 정규화)
transform = transforms.Compose([
    transforms.ToTensor(),
])

# 학습 및 테스트 데이터셋 로드
train_dataset = datasets.FashionMNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.FashionMNIST(root='./data', train=False, transform=transform, download=True)

# 데이터 로더 생성
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

---

# CNN

## 05-001 2D 컨볼루션 출력 크기 계산 함수 구현
- 이 실습에서는 2D 컨볼루션 레이어의 출력 크기를 계산하는 함수를 구현합니다. 입력 크기, 커널 크기, 스트라이드, 패딩을 매개변수로 받아 컨볼루션 계산 공식을 사용하여 출력 높이와 너비를 반환합니다. 이 함수는 CNN 모델 설계 시 출력 크기를 예측하는 데 유용합니다.

In [ ]:
def conv2d_output_size(input_size, kernel_size, stride=1, padding=0):
    height, width = input_size

    # Convolution 공식 적용
    out_height = (height + 2 * padding - kernel_size) // stride + 1
    out_width = (width + 2 * padding - kernel_size) // stride + 1

    return out_height, out_width

- 2D 컨볼루션의 출력 크기를 계산하는 함수를 구현하는 과정을 실습합니다. 이 함수는 신경망 설계 시 중요한 역할을 합니다.

## 05-002 컨볼루션 출력 크기 계산
- 이 실습에서는 앞서 구현한 conv2d_output_size() 함수를 사용하여 컨볼루션 레이어의 출력 크기를 계산합니다. 입력으로 28x28 크기의 이미지와 3x3 커널, 스트라이드 1, 패딩 0을 설정하여 출력 feature map의 크기를 확인합니다.

In [ ]:
# 입력 크기 (Height, Width), 커널 크기, 스트라이드, 패딩
output_size = conv2d_output_size((28, 28), 3, 1, 0)
print(f"Output feature map size: {output_size}")

Output feature map size: (26, 26)


- 입력 이미지와 커널의 크기를 사용하여 컨볼루션의 출력 크기를 계산하고 확인하는 과정을 실습합니다. 이를 통해 CNN 모델의 구조를 이해하는 데 도움이 됩니다.

## 05-003 패딩을 포함한 컨볼루션 출력 크기 계산
- 이 실습에서는 conv2d_output_size() 함수를 사용하여 패딩을 포함한 컨볼루션 레이어의 출력 크기를 계산합니다. 입력으로 28x28 크기의 이미지와 3x3 커널, 스트라이드 1, 패딩 1을 설정하여 출력 feature map의 크기를 확인합니다. 이 과정은 CNN 설계 시 출력 크기를 예측하는 데 중요합니다.

In [ ]:
# 입력 크기 (Height, Width), 커널 크기, 스트라이드, 패딩
output_size = conv2d_output_size((28, 28), 3, 1, 1)
print(f"Output feature map size: {output_size}")

Output feature map size: (28, 28)


- 패딩을 포함하여 컨볼루션의 출력 크기를 계산하고 확인하는 과정을 실습합니다. 패딩의 효과를 이해하는 데 도움이 됩니다.

## 05-004 스트라이드를 포함한 컨볼루션 출력 크기 계산
- 이 실습에서는 conv2d_output_size() 함수를 사용하여 스트라이드를 포함한 컨볼루션 레이어의 출력 크기를 계산합니다. 입력으로 28x28 크기의 이미지와 3x3 커널, 스트라이드 2, 패딩 0을 설정하여 출력 feature map의 크기를 확인합니다. 이 과정은 CNN 모델 설계 시 중요한 요소입니다.

In [ ]:
# 입력 크기 (Height, Width), 커널 크기, 스트라이드, 패딩
output_size = conv2d_output_size((28, 28), 3, 2, 0)
print(f"Output feature map size: {output_size}")

Output feature map size: (13, 13)


- 스트라이드를 포함하여 컨볼루션의 출력 크기를 계산하고 확인하는 과정을 실습합니다. 스트라이드의 효과를 이해하는 데 도움이 됩니다.

## 05-005 스트라이드 및 패딩을 포함한 컨볼루션 출력 크기 계산
- 이 실습에서는 conv2d_output_size() 함수를 사용하여 스트라이드와 패딩을 모두 포함한 컨볼루션 레이어의 출력 크기를 계산합니다. 입력으로 28x28 크기의 이미지와 3x3 커널, 스트라이드 2, 패딩 1을 설정하여 출력 feature map의 크기를 확인합니다. 이 과정은 CNN 설계 시 중요한 요소입니다.

In [ ]:
# 입력 크기 (Height, Width), 커널 크기, 스트라이드, 패딩
output_size = conv2d_output_size((28, 28), 3, 2, 1)
print(f"Output feature map size: {output_size}")

Output feature map size: (14, 14)


- 스트라이드와 패딩을 포함하여 컨볼루션의 출력 크기를 계산하고 확인하는 과정을 실습합니다. 이를 통해 CNN의 동작을 더 깊이 이해할 수 있습니다.

## 05-006 CNN 모델 클래스 정의
- 이 실습에서는 CNN(Convolutional Neural Network) 모델을 정의합니다. 두 개의 컨볼루션 레이어, 각 레이어 뒤에 배치 정규화 및 드롭아웃을 적용하고, 최종적으로 두 개의 완전 연결 레이어를 사용하여 FashionMNIST 데이터셋의 분류 문제를 해결합니다. summary() 함수를 통해 모델 구조를 확인합니다.

In [ ]:
# CNN 모델 정의
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        # 첫 번째 컨볼루션 레이어
        # 입력 채널: 1 (흑백 이미지), 출력 채널: 16, 커널 크기: 3x3, 패딩: 1
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding=1)

        # 두 번째 컨볼루션 레이어
        # 입력 채널: 16, 출력 채널: 32, 커널 크기: 3x3, 패딩: 1
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)

        # MaxPool 레이어 (다운샘플링)
        # 커널 크기: 2x2, 스트라이드: 2
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        # 첫 번째 완전 연결 (Fully Connected) 레이어
        # 입력 크기: 32 * 7 * 7 (Conv2d 출력을 펼친 크기), 출력 크기: 128
        self.fc1 = nn.Linear(32 * 7 * 7, 128)

        # 두 번째 완전 연결 레이어
        # 입력 크기: 128, 출력 크기: 10 (클래스 개수)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        # 입력 데이터 크기: (batch_size, 1, 28, 28)

        # 첫 번째 Conv + ReLU + MaxPool
        # Conv 후 크기: (batch_size, 16, 28, 28)
        # MaxPool 후 크기: (batch_size, 16, 14, 14)
        x = self.pool(torch.relu(self.conv1(x)))

        # 두 번째 Conv + ReLU + MaxPool
        # Conv 후 크기: (batch_size, 32, 14, 14)
        # MaxPool 후 크기: (batch_size, 32, 7, 7)
        x = self.pool(torch.relu(self.conv2(x)))

        # Flatten: Conv 출력을 1차원 벡터로 펼침
        # Flatten 후 크기: (batch_size, 32 * 7 * 7)
        x = x.reshape(-1, 32 * 7 * 7)

        # 첫 번째 Fully Connected + ReLU
        # 출력 크기: (batch_size, 128)
        x = torch.relu(self.fc1(x))

        # 두 번째 Fully Connected (출력층)
        # 출력 크기: (batch_size, 10)
        x = self.fc2(x)

        return x

# 모델 요약 출력
model = CNN().to(device)
summary(model, input_size=(64, 1, 28, 28))

Layer (type:depth-idx)                   Output Shape              Param #
CNN                                      [64, 10]                  --
├─Conv2d: 1-1                            [64, 16, 28, 28]          160
├─MaxPool2d: 1-2                         [64, 16, 14, 14]          --
├─Conv2d: 1-3                            [64, 32, 14, 14]          4,640
├─MaxPool2d: 1-4                         [64, 32, 7, 7]            --
├─Linear: 1-5                            [64, 128]                 200,832
├─Linear: 1-6                            [64, 10]                  1,290
Total params: 206,922
Trainable params: 206,922
Non-trainable params: 0
Total mult-adds (M): 79.17
Input size (MB): 0.20
Forward/backward pass size (MB): 9.70
Params size (MB): 0.83
Estimated Total Size (MB): 10.73

- CNN 모델 클래스를 구현하고, 요약 정보를 통해 모델의 구조와 파라미터 수를 확인하는 과정을 실습합니다. CNN은 이미지 분류 문제에 효과적인 신경망 구조입니다.

## 05-007 손실 함수 및 옵티마이저 설정과 학습 수행
- 이 실습에서는 모델 학습을 위한 손실 함수와 옵티마이저를 설정합니다. nn.CrossEntropyLoss()는 다중 클래스 분류 문제에 적합한 손실 함수로, 각 클래스의 확률을 기반으로 손실을 계산합니다. Adam 옵티마이저는 학습률(lr)을 0.0001로 설정하여 모델의 파라미터를 업데이트합니다. 이후, train_and_evaluate() 함수를 호출하여 모델을 학습하고 평가합니다.

In [ ]:
# 손실 함수와 옵티마이저 설정
criterion = nn.CrossEntropyLoss()  # 다중 클래스 분류를 위한 손실 함수
optimizer = optim.Adam(model.parameters(), lr=0.0001)  # Adam 옵티마이저

train_losses, train_accuracies, test_losses, test_accuracies = train_and_evaluate(
    model, train_loader, test_loader, criterion, optimizer, num_epochs=10, device=device
)

- 손실 함수와 옵티마이저를 설정한 후, 모델을 학습하고 평가하는 과정을 실습합니다. 이 과정은 신경망 모델의 성능을 최적화하는 데 필수적입니다.

## 05-008 JAEN 패키지에서 CNN 모델 불러오기
- 이 실습에서는 JAEN 패키지에서 제공하는 CNN 모델을 불러오고, 사전 학습된(pretrained) 가중치를 사용하여 모델을 초기화하는 방법을 학습합니다. summary() 함수를 사용하여 모델의 구조와 파라미터 수를 확인하여 모델이 올바르게 로드되었는지 검증합니다.

In [ ]:
# JAEN 패키지에서 CNN 모델 가져오기
from JAEN.models import CNNModel

# CNN 모델 불러오기 (pretrained=True)
model = CNNModel(pretrained=True)

# 모델 정보 확인
summary(model, (64, 1, 28, 28))

- JAEN 패키지에서 CNN 모델을 불러오고, 모델 정보를 요약하여 확인하는 과정을 실습합니다. 이는 사전 학습된 모델을 사용할 때 유용합니다.

## 05-009 기존 Conv Block 동결
- 이 실습에서는 사전 학습된 CNN 모델의 일부 레이어(Conv Block)의 파라미터를 동결하여 학습 중에 업데이트되지 않도록 설정하는 방법을 학습합니다. 이는 transfer learning 기법의 일환으로, 특정 레이어의 가중치를 고정하여 새로운 데이터셋에 대해 학습할 때 유용합니다.

In [ ]:
# 기존 Conv Block 동결
for param in model.conv_layers.parameters():
    param.requires_grad = False

- CNN 모델의 기존 Conv Block을 동결하여 해당 레이어의 파라미터가 학습되지 않도록 설정하는 과정을 실습합니다. 이는 전이 학습 시 일반적인 기법입니다.

## 05-010 새로운 Fully Connected Block 설정
- 이 실습에서는 CNN 모델의 Fully Connected Block을 새롭게 설정합니다. nn.Sequential()을 사용하여 새로운 은닉층과 드롭아웃 레이어를 추가하고, 출력층을 정의합니다. 모델을 디바이스(GPU 또는 CPU)로 이동한 후, summary() 함수를 사용하여 모델의 구조를 확인합니다.

In [ ]:
# 새로운 Fully Connected Block 설정
model.fc_layers = nn.Sequential(
    nn.Linear(32 * 7 * 7, 64),  # 첫 번째 은닉층
    nn.ReLU(),
    nn.Dropout(p=0.5),           # 드롭아웃 추가
    nn.Linear(64, 10)           # 출력층 (활성화 함수 없음)
)

# 디바이스 설정 (GPU 또는 CPU)
model = model.to(device)
summary(model, input_size=(64, 1, 28, 28))

- 새로운 Fully Connected Block을 설정하고, 모델의 구조와 파라미터 수를 요약하여 확인하는 과정을 실습합니다. 이는 모델 구조를 조정하고 최적화하는 데 중요한 단계입니다.

## 05-011 손실 함수 및 최적화 도구 정의와 학습 수행
- 이 실습에서는 손실 함수와 최적화 도구를 정의합니다. nn.CrossEntropyLoss()는 다중 클래스 분류에 적합한 손실 함수이며, Adam 옵티마이저는 requires_grad가 True인 파라미터만 업데이트하도록 설정합니다. 이후, train_and_evaluate() 함수를 호출하여 모델을 학습하고 평가합니다.

In [ ]:
# 손실 함수와 최적화 도구 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.0001)

train_losses, train_accuracies, test_losses, test_accuracies = train_and_evaluate(
    model, train_loader, test_loader, criterion, optimizer, num_epochs=10, device=device
)

- 손실 함수와 최적화 도구를 설정한 후, 모델을 학습하고 평가하는 과정을 실습합니다. 이 과정은 신경망 모델의 성능을 최적화하는 데 필수적입니다.

## 05-012 마지막 두 Conv 레이어만 학습하도록 설정
- 이 실습에서는 CNN 모델에서 마지막 두 컨볼루션 레이어의 파라미터만 학습하도록 설정합니다. named_parameters() 메서드를 사용하여 각 레이어의 이름과 파라미터를 반복하고, 특정 레이어에 대해서만 requires_grad를 True로 설정합니다. 이를 통해 모델의 특정 부분만 학습하여 과적합을 방지하고 성능을 최적화할 수 있습니다.

In [ ]:
# 마지막 두 Conv 레이어만 학습하도록 설정
for name, p in model.conv_layers.named_parameters():
    if name in ['5.weight', '5.bias', '7.weight', '7.bias']:
        p.requires_grad = True

- CNN 모델의 마지막 두 Conv 레이어만 학습하도록 설정하는 과정을 실습합니다. 이는 전이 학습 시 효과적인 접근 방식입니다.

## 05-013 손실 함수 및 최적화 도구 정의와 학습 수행
- 이 실습에서는 손실 함수와 최적화 도구를 정의합니다. nn.CrossEntropyLoss()는 다중 클래스 분류에 적합한 손실 함수이며, Adam 옵티마이저는 requires_grad가 True인 파라미터만 업데이트하도록 설정합니다. 이후, train_and_evaluate() 함수를 호출하여 모델을 학습하고 평가합니다.

In [ ]:
# 손실 함수와 최적화 도구 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.0001)

train_losses, train_accuracies, test_losses, test_accuracies = train_and_evaluate(
    model, train_loader, test_loader, criterion, optimizer, num_epochs=10, device=device
)

- 손실 함수와 최적화 도구를 설정한 후, 모델을 학습하고 평가하는 과정을 실습합니다. 이 과정은 신경망 모델의 성능을 최적화하는 데 필수적입니다.

---

## 연습문제-05-001 (배치, 3, 32, 32) 입력을 받는 CNN 모델 설계 (Sequential 방식)
- 이 실습에서는 PyTorch Sequential 방식을 사용하여 (배치, 3, 32, 32) 형태의 이미지 입력을 받는 CNN 모델을 설계하는 방법을 학습합니다. 이 CNN 모델은 세 개의 합성곱 층을 사용하며, 각 층의 뒤에는 ReLU 활성화 함수와 Max Pooling을 적용합니다. 모델은 64개의 필터를 가진 마지막 합성곱 층에서 특징을 추출하고, 이를 완전히 연결된 층으로 전달합니다. 마지막 출력은 10개의 클래스에 대한 확률값을 나타내며, 소프트맥스 활성화 함수를 사용하여 이를 계산합니다.

학습할 때, 이 모델은 (배치, 3, 32, 32) 형태의 컬러 이미지를 입력으로 받아 분류 작업을 수행할 수 있습니다. 각 이미지의 높이와 너비는 32x32 픽셀이며, 채널은 RGB(3개)로 구성됩니다.

In [ ]:
import torch
import torch.nn as nn

model = nn.Sequential(
            # 첫 번째 합성곱층과 풀링층
            nn.Conv2d(3, 16, kernel_size=3, padding=1),  # 입력 채널 3, 출력 채널 16, 커널 크기 3x3, 패딩 1
            nn.ReLU(),  # 활성화 함수
            nn.MaxPool2d(kernel_size=2, stride=2),  # 풀링층: 출력 크기를 절반으로 축소 (출력 크기: (16, 16))

            # 두 번째 합성곱층과 풀링층
            nn.Conv2d(16, 32, kernel_size=3, padding=1),  # 입력 채널 16, 출력 채널 32
            nn.ReLU(),  # 활성화 함수
            nn.MaxPool2d(kernel_size=2, stride=2),  # 출력 크기를 절반으로 축소 (출력 크기: (8, 8))

            # 세 번째 합성곱층과 풀링층
            nn.Conv2d(32, 64, kernel_size=3, padding=1),  # 입력 채널 32, 출력 채널 64
            nn.ReLU(),  # 활성화 함수
            nn.MaxPool2d(kernel_size=2, stride=2),  # 출력 크기를 절반으로 축소 (출력 크기: (4, 4))

            # Flatten 레이어
            nn.Flatten(),  # (64, 4, 4) 형태의 출력을 일차원 벡터로 변환

            # 첫 번째 완전 연결층
            nn.Linear(64 * 4 * 4, 128),  # Conv 레이어 출력을 128차원 출력으로 변환
            nn.ReLU(),  # 활성화 함수

            # 최종 출력층
            nn.Linear(128, 10)  # 10개의 클래스로 분류
        )

model

Sequential(
  (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU()
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): ReLU()
  (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (7): ReLU()
  (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (9): Flatten(start_dim=1, end_dim=-1)
  (10): Linear(in_features=1024, out_features=128, bias=True)
  (11): ReLU()
  (12): Linear(in_features=128, out_features=10, bias=True)
)

- (배치, 3, 32, 32) 형태의 입력을 받는 CNN 모델을 Sequential 방식으로 설계하여 이미지 분류 문제를 해결하는 방법을 실습합니다. CNN의 기본 구조와 작동 원리에 대해 익히며, 합성곱, ReLU 활성화 함수, Max Pooling, 그리고 완전 연결 층의 역할을 이해합니다.

## 연습문제-05-002 (배치, 3, 224, 224) 입력을 받는 CNN 모델 설계 (Sequential 방식)
- 이 실습에서는 PyTorch Sequential 방식을 사용하여 (배치, 3, 224, 224) 형태의 이미지 입력을 받는 CNN 모델을 설계하는 방법을 학습합니다. 이 모델은 더 큰 이미지(224x224 픽셀)를 처리할 수 있도록 설계되었으며, 네 개의 합성곱 층을 사용하여 이미지에서 점차적으로 더 추상적인 특징을 추출합니다. 각 합성곱 층 뒤에는 ReLU 활성화 함수와 Max Pooling을 적용하여 공간 정보를 줄이면서도 유용한 특징을 남깁니다. 마지막 완전 연결 층에서 추출된 특징을 기반으로 최종적으로 2개의 클래스를 분류합니다.

이 모델은 매우 큰 이미지(224x224)를 처리할 수 있도록 최적화되어 있으며, 이러한 이미지들은 일반적으로 고해상도 이미지를 나타냅니다. 이미지 분류 문제에서 이러한 모델은 큰 특징 맵을 효과적으로 다룰 수 있는 방법을 제공합니다.

In [ ]:
import torch
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        # 첫 번째 합성곱층과 풀링층
        self.conv1 = nn.Conv2d(3, 256, kernel_size=3, padding=1)  # 입력 채널 3, 출력 채널 256, 커널 크기 3x3, 패딩 1
        self.relu1 = nn.ReLU()  # 활성화 함수
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)  # 풀링층: 출력 크기를 (112, 112)로 축소

        # 두 번째 합성곱층과 풀링층
        self.conv2 = nn.Conv2d(256, 128, kernel_size=3, padding=1)  # 입력 채널 256, 출력 채널 128
        self.relu2 = nn.ReLU()  # 활성화 함수
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)  # 출력 크기를 (56, 56)으로 축소

        # 세 번째 합성곱층과 풀링층
        self.conv3 = nn.Conv2d(128, 64, kernel_size=3, padding=1)  # 입력 채널 128, 출력 채널 64
        self.relu3 = nn.ReLU()  # 활성화 함수
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)  # 출력 크기를 (28, 28)으로 축소

        # 네 번째 합성곱층과 풀링층
        self.conv4 = nn.Conv2d(64, 32, kernel_size=3, padding=1)  # 입력 채널 64, 출력 채널 32
        self.relu4 = nn.ReLU()  # 활성화 함수
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)  # 출력 크기를 (14, 14)로 축소

        # Flatten 레이어
        self.flatten = nn.Flatten()  # (32, 14, 14) 형태의 출력을 일차원 벡터로 변환

        # 첫 번째 완전 연결층
        self.fc1 = nn.Linear(32 * 14 * 14, 256)  # Conv 레이어 출력을 낮은 차원으로 줄임
        self.relu_fc1 = nn.ReLU()  # 활성화 함수

        # 두 번째 완전 연결층
        self.fc2 = nn.Linear(256, 2)  # 두 개의 클래스로 최종 분류

    def forward(self, x):
        # 첫 번째 합성곱층과 풀링층
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)

        # 두 번째 합성곱층과 풀링층
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)

        # 세 번째 합성곱층과 풀링층
        x = self.conv3(x)
        x = self.relu3(x)
        x = self.pool3(x)

        # 네 번째 합성곱층과 풀링층
        x = self.conv4(x)
        x = self.relu4(x)
        x = self.pool4(x)

        # Flatten 레이어
        x = self.flatten(x)

        # 첫 번째 완전 연결층
        x = self.fc1(x)
        x = self.relu_fc1(x)

        # 두 번째 완전 연결층 (최종 출력)
        x = self.fc2(x)

        return x

model = CNN()
model

CNN(
  (conv1): Conv2d(3, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu1): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(256, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu3): ReLU()
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu4): ReLU()
  (pool4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=6272, out_features=256, bias=True)
  (relu_fc1): ReLU()
  (fc2): Linear(in_features=256, out_features=2, bias=True)
)

- (배치, 3, 224, 224) 형태의 입력을 받는 CNN 모델을 Sequential 방식으로 설계하여 고해상도 이미지 분류 문제를 해결하는 방법을 실습합니다. 더 깊은 네트워크 구조를 통해 고해상도 이미지를 처리하며, 각 층이 추출하는 특징과 네트워크의 학습 방식에 대해 이해합니다.

---

In [ ]:
# 필요 도구 import
import torch
import torch.nn as nn
import torch.optim as optim
from torchinfo import summary
from torch.utils.data import DataLoader, Dataset
from JAEN.utils import train_and_evaluate

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

# 텍스트 데이터 - 토큰화

## 06-001 Tokenizer 불러오기
- 이 실습에서는 Hugging Face의 Transformers 라이브러리를 사용하여 BERT 모델의 토크나이저를 불러오는 방법을 학습합니다. AutoTokenizer를 사용하여 'bert-base-cased' 모델에 맞는 토크나이저를 자동으로 가져오며, 이 토크나이저는 입력 텍스트를 BERT 모델에서 처리할 수 있는 형식으로 변환하는 데 사용됩니다.

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

- BERT 모델에 사용할 토크나이저를 불러오는 과정을 실습합니다. 이는 NLP 작업에서 텍스트를 준비하는 데 중요한 단계입니다.

## 06-002 토큰화 실습
- 이 실습에서는 불러온 BERT 토크나이저를 사용하여 주어진 텍스트 'AI Essential'을 토크나이징하는 방법을 학습합니다. 토크나이저는 입력 문자열을 BERT 모델이 이해할 수 있는 형식으로 변환하며, 각 단어를 고유한 ID로 매핑합니다.

In [ ]:
tokenizer("AI Essential")

{'input_ids': [101, 19016, 26768, 102], 'token_type_ids': [0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1]}

- BERT 토크나이저를 사용하여 텍스트를 토크나이즈하는 과정을 실습합니다. 이는 NLP 모델에서 입력 데이터를 준비하는 데 필수적인 과정입니다.

## 06-003 토큰화된 텍스트 확인
- 이 실습에서는 BERT 토크나이저를 사용하여 텍스트 'AI Essential'을 토큰화한 결과를 확인합니다. tokenizer.tokenize() 메서드를 통해 입력 텍스트를 개별 토큰으로 분리하며, 각 토큰은 BERT 모델이 처리할 수 있는 형태로 변환됩니다.

In [ ]:
tokens = tokenizer.tokenize("AI Essential")
tokens

['AI', 'Essential']

- 주어진 텍스트를 BERT 토크나이저를 사용하여 토큰화하고, 생성된 토큰을 확인하는 과정을 실습합니다. 이는 NLP 모델에서 데이터를 준비하는 중요한 단계입니다.

## 06-004 토큰을 ID로 변환
- 이 실습에서는 BERT 토크나이저를 사용하여 토큰화된 텍스트를 고유한 정수 ID로 변환하는 방법을 학습합니다. tokenizer.convert_tokens_to_ids() 메서드를 통해 각 토큰을 모델에서 사용하는 입력 형태로 변환합니다.

In [ ]:
ids = tokenizer.convert_tokens_to_ids(tokens)
ids

[19016, 26768]

- 토큰화된 텍스트를 BERT 모델의 입력으로 사용할 수 있도록 정수 ID로 변환하는 과정을 실습합니다. 이는 NLP 모델에 데이터를 입력하는 데 필수적인 과정입니다.

## 06-005 토큰 ID를 텍스트로 디코딩
- 이 실습에서는 BERT 토크나이저를 사용하여 토큰 ID를 다시 원래의 텍스트로 디코딩하는 방법을 학습합니다. tokenizer.decode() 메서드를 사용하여 ID를 입력하면, 모델이 이해할 수 있는 형태의 원본 문자열로 복원됩니다.

In [ ]:
tokenizer.decode(ids)

'AI Essential'

- 토큰 ID를 사용하여 원래의 텍스트로 디코딩하는 과정을 실습합니다. 이는 NLP 작업에서 데이터를 확인하고 검증하는 데 유용합니다.

## 06-006 임베딩 모듈 생성 및 사용
- 이 실습에서는 PyTorch의 nn.Embedding 모듈을 사용하여 정수 인덱스를 고차원 벡터로 변환하는 방법을 학습합니다. 임베딩 레이어를 생성하고, 입력 데이터로 주어진 정수 인덱스를 벡터로 변환합니다. 이 과정은 자연어 처리(NLP) 작업에서 단어 또는 토큰을 임베딩하는 데 사용됩니다.

In [ ]:
import torch
import torch.nn as nn

# 임베딩 모듈 생성 (정수 인덱스 10개, 각 인덱스는 5차원의 벡터로 매핑)
embedding_layer = nn.Embedding(num_embeddings=10, embedding_dim=5)

# 임베딩에 사용할 예시 입력 데이터 (정수 인덱스)
# 여기서 [2, 5, 7]은 단어나 토큰에 해당한다고 가정
input_data = torch.tensor([2, 5, 7])

# 임베딩 레이어에 입력 데이터를 전달하여 벡터 변환
embedded_output = embedding_layer(input_data)

print("입력 데이터 (정수 인덱스):", input_data)
print("임베딩 결과 (벡터):\n", embedded_output)

입력 데이터 (정수 인덱스): tensor([2, 5, 7])
임베딩 결과 (벡터):
 tensor([[ 0.0483,  0.6007, -0.4233,  1.1709, -0.7364],
        [-0.4178, -0.4323, -1.1088, -1.0098,  1.2270],
        [ 0.7314,  2.0637, -0.0360, -0.5074, -0.0572]],
       grad_fn=<EmbeddingBackward0>)


- 임베딩 모듈을 생성하고, 정수 인덱스를 벡터로 변환하는 과정을 실습합니다. 이는 NLP 모델에서 단어 표현을 얻는 데 필수적인 단계입니다.

## 06-007 코퍼스 및 레이블 정의
- 이 실습에서는 간단한 문장 리스트(corpus)와 각 문장에 대한 레이블(y)을 정의합니다. y는 부정(0)과 긍정(1)으로 이루어져 있으며, 각 문장의 감정 평가를 나타냅니다. 이러한 데이터는 감정 분석 모델을 학습시키는 데 사용됩니다.

In [ ]:
corpus = [
    'very good nice quality',
    'stop lying',
    'ugly terrible',
    'excellent work',
    'adorable lovely',
    'bad',
    'great nice'
]
y = torch.FloatTensor([1, 0, 0, 1, 1, 0, 1]).view(-1, 1)  # 0: 부정, 1: 긍정

- 감정 분석을 위한 문장 목록과 레이블을 정의하는 과정을 실습합니다. 이는 NLP 모델 학습을 위한 데이터 준비의 기초 단계입니다.

## 06-008 GPT2 토크나이저로 문장 변환
- 이 실습에서는 GPT2 모델의 토크나이저를 사용하여 문장 목록(corpus)을 토큰화합니다. AutoTokenizer를 통해 사전 학습된 토크나이저를 불러오고, clean_up_tokenization_spaces=True 옵션을 통해 불필요한 공백을 제거합니다. 변환된 시퀀스는 각 문장을 토큰 ID의 리스트로 표현합니다.

In [ ]:
from transformers import AutoTokenizer

# GPT2 토크나이저 불러오기
tokenizer = AutoTokenizer.from_pretrained(
    'gmteacher/simple-word-tokenizer',
    clean_up_tokenization_spaces=True)
seqs = tokenizer(corpus)['input_ids']
seqs  # 변환된 시퀀스

tokenizer_config.json:   0%|          | 0.00/560 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/977 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

[[14, 6, 2, 10], [11, 9], [13, 12], [5, 15], [3, 8], [4], [7, 2]]

- GPT2 토크나이저를 사용하여 문장 목록을 토큰화하고, 변환된 시퀀스를 확인하는 과정을 실습합니다. 이는 NLP 모델의 입력 데이터를 준비하는 데 필수적인 단계입니다.

## 06-009 토큰 ID를 단어로 복원
- 이 실습에서는 GPT2 토크나이저를 사용하여 토큰 ID를 원래의 단어로 복원합니다. tokenizer.decode() 메서드를 사용하여 각 시퀀스를 입력하면, 모델이 이해할 수 있는 형태의 원본 문자열로 변환됩니다.

In [ ]:
# 시퀀스를 단어로 복원
[tokenizer.decode(seq) for seq in seqs]

['very good nice quality',
 'stop lying',
 'ugly terrible',
 'excellent work',
 'adorable lovely',
 'bad',
 'great nice']

- 토큰 ID를 사용하여 원래의 단어로 복원하는 과정을 실습합니다. 이는 NLP 작업에서 데이터를 확인하고 검증하는 데 유용합니다.

## 06-010 시퀀스를 텐서로 변환하고 패딩
- 이 실습에서는 토큰 ID로 변환된 시퀀스들을 PyTorch 텐서로 변환한 후, pad_sequence() 함수를 사용하여 패딩을 적용합니다. 패딩을 통해 각 시퀀스의 길이를 동일하게 맞추어 배치 처리를 용이하게 합니다. batch_first=True로 설정하여 배치가 첫 번째 차원에 오도록 합니다.

In [ ]:
from torch.nn.utils.rnn import pad_sequence

# 각 시퀀스를 텐서로 변환
seqs = [torch.tensor(seq) for seq in seqs]

# 패딩
x = pad_sequence(seqs, batch_first=True)
x

<ipython-input-18-531871042a50>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  seqs = [torch.tensor(seq) for seq in seqs]


tensor([[14,  6,  2, 10],
        [11,  9,  0,  0],
        [13, 12,  0,  0],
        [ 5, 15,  0,  0],
        [ 3,  8,  0,  0],
        [ 4,  0,  0,  0],
        [ 7,  2,  0,  0]])

- 각 시퀀스를 텐서로 변환하고, 패딩을 적용하여 배치 처리를 위한 텐서를 준비하는 과정을 실습합니다. 이는 시퀀스 데이터 처리에서 매우 중요한 단계입니다.

## 06-011 텍스트 분류기 모델 정의
- 이 실습에서는 텍스트 분류를 위한 신경망 모델을 정의합니다. TextClassifier 클래스는 입력 데이터를 임베딩 레이어를 통해 고차원 벡터로 변환하고, 평탄화(flatten)한 후 완전 연결 레이어를 사용하여 클래스를 예측합니다. 마지막으로, Sigmoid 활성화 함수를 사용하여 출력을 변환합니다.

In [ ]:
class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, seq_len, num_class):
        super().__init__()
        # 임베딩 계층 추가
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.flat = nn.Flatten()
        self.fc = nn.Linear(embed_dim*seq_len, num_class)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.embedding(x)
        out = self.flat(out)
        out = self.fc(out)
        out = self.sigmoid(out)
        return out

- 텍스트 분류를 위한 신경망 모델을 정의하는 과정을 실습합니다. 이 모델은 NLP 작업에서 입력 데이터를 분류하는 데 사용됩니다.

## 06-012 텍스트 분류기 모델 인스턴스 생성 및 출력 테스트
- 이 실습에서는 TextClassifier 모델의 인스턴스를 생성하고, 입력 데이터에 대한 출력 결과의 형태를 확인합니다. torch.manual_seed(0)으로 랜덤 시드를 설정하여 결과의 재현성을 보장합니다. 모델의 출력이 올바르게 형성되는지를 테스트합니다.

In [ ]:
torch.manual_seed(0)
# 모델 인스턴스 생성
model = TextClassifier(tokenizer.vocab_size, 2, x.shape[1], 1).to(device)

# 출력 테스트
output = model(x[:1].to(device))
output.shape

torch.Size([1, 1])

- 텍스트 분류기 모델을 인스턴스화하고, 입력 데이터를 사용하여 출력의 형태를 확인하는 과정을 실습합니다. 이는 모델이 기대한 대로 작동하는지 검증하는 데 중요합니다.

## 06-013 장치 적용
- 이 실습에서는 입력 데이터 x와 타겟 데이터 y를 설정한 장치(GPU 또는 CPU)로 이동합니다. PyTorch에서는 모델과 데이터가 동일한 장치에 있어야 연산이 가능하므로, 이를 통해 학습 및 추론을 위한 준비를 합니다.

In [ ]:
# 장치 적용
x = x.to(device)
y = y.to(device)

- 입력 데이터와 타겟 데이터를 장치에 적용하는 과정을 실습합니다. 이는 PyTorch에서 효율적인 계산을 위한 필수적인 단계입니다.

## 06-014 모델 학습 및 손실 계산
- 이 실습에서는 정의한 텍스트 분류기 모델을 학습시키고 손실을 계산하는 과정을 구현합니다. BCELoss를 손실 함수로 사용하고, Adam 옵티마이저를 통해 모델의 파라미터를 업데이트합니다. 지정된 최대 에폭 수만큼 모델을 학습시키며, 주기적으로 손실 값을 출력합니다.

In [ ]:
loss_fn = nn.BCELoss()  # 손실 함수
optimizer = optim.Adam(model.parameters())  # 최적화 도구(optimizer)

epochs = 5000  # 최대 에폭 지정
results = {'cost':[]}

model.train()  # 학습 모드 설정

for epoch in range(epochs):
    h = model(x)  # 예측 값 생성(추론)
    loss = loss_fn(h, y)  # 손실 계산

    optimizer.zero_grad()  # 미분 값 초기화
    loss.backward()  # 역전파(미분 계산)
    optimizer.step()  # 업데이트 진행

    results['cost'].append(loss.item())

    if epoch % 1000 == 0:
        print(f'epoch: {epoch:4d}, cost: {results["cost"][-1]:.10f}')

- 모델을 학습시키고 손실을 계산하여 업데이트하는 과정을 실습합니다. 학습 과정의 손실 변화를 모니터링하는 것은 모델의 성능을 이해하는 데 중요합니다.

## 06-015 예측 값을 클래스로 변환
- 이 실습에서는 모델의 예측 값을 시그모이드 결과를 기반으로 클래스로 변환하는 방법을 학습합니다. h.view(-1)로 예측 결과를 평탄화한 후, 0.5를 기준으로 이진 클래스로 변환합니다. 실제 값과 예측 값을 출력하여 모델의 성능을 평가합니다.

In [ ]:
# 예측 값 생성
h = model(x)

# 예측 값(시그모이드 값)을 클래스로 변환
pred = (h.view(-1).detach().cpu() > 0.5).to(torch.float32)

print(f'실제 값: {y.view(-1)}')
print(f'예측 값: {pred}')

- 모델의 예측 값을 클래스 형태로 변환하고, 실제 값과 비교하여 출력하는 과정을 실습합니다. 이는 모델의 예측 정확도를 확인하는 데 중요합니다.

---

# RNN

## 06-016 코퍼스 및 레이블 정의
- 이 실습에서는 간단한 문장 리스트(corpus)와 각 문장에 대한 레이블(y)을 정의합니다. y는 부정(0)과 긍정(1)으로 이루어져 있으며, 각 문장의 감정 평가를 나타냅니다. 이러한 데이터는 감정 분석 모델을 학습시키는 데 사용됩니다.

In [ ]:
corpus = [
    'very good nice quality',
    'stop lying',
    'ugly terrible',
    'excellent work',
    'adorable lovely',
    'bad',
    'great nice'
]
y = torch.FloatTensor([1, 0, 0, 1, 1, 0, 1]).view(-1, 1)  # 0: 부정, 1: 긍정

- 감정 분석을 위한 문장 목록과 레이블을 정의하는 과정을 실습합니다. 이는 NLP 모델 학습을 위한 데이터 준비의 기초 단계입니다.

## 06-017 GPT2 토크나이저로 문장 변환
- 이 실습에서는 GPT2 모델의 토크나이저를 사용하여 문장 목록(corpus)을 토큰화합니다. AutoTokenizer를 통해 사전 학습된 토크나이저를 불러오고, clean_up_tokenization_spaces=True 옵션을 통해 불필요한 공백을 제거합니다. 변환된 시퀀스는 각 문장을 토큰 ID의 리스트로 표현합니다.

In [ ]:
from transformers import AutoTokenizer

# GPT2 토크나이저 불러오기
tokenizer = AutoTokenizer.from_pretrained(
    'gmteacher/simple-word-tokenizer',
    clean_up_tokenization_spaces=True)
seqs = tokenizer(corpus)['input_ids']
seqs  # 변환된 시퀀스

- GPT2 토크나이저를 사용하여 문장 목록을 토큰화하고, 변환된 시퀀스를 확인하는 과정을 실습합니다. 이는 NLP 모델의 입력 데이터를 준비하는 데 필수적인 단계입니다.

## 06-018 토큰 ID를 단어로 복원
- 이 실습에서는 GPT2 토크나이저를 사용하여 토큰 ID를 원래의 단어로 복원합니다. tokenizer.decode() 메서드를 사용하여 각 시퀀스를 입력하면, 모델이 이해할 수 있는 형태의 원본 문자열로 변환됩니다.

In [ ]:
# 시퀀스를 단어로 복원
[tokenizer.decode(seq) for seq in seqs]

- 토큰 ID를 사용하여 원래의 단어로 복원하는 과정을 실습합니다. 이는 NLP 작업에서 데이터를 확인하고 검증하는 데 유용합니다.

## 06-019 시퀀스를 텐서로 변환하고 패딩
- 이 실습에서는 토큰 ID로 변환된 시퀀스들을 PyTorch 텐서로 변환한 후, pad_sequence() 함수를 사용하여 패딩을 적용합니다. 패딩을 통해 각 시퀀스의 길이를 동일하게 맞추어 배치 처리를 용이하게 합니다. batch_first=True로 설정하여 배치가 첫 번째 차원에 오도록 합니다.

In [ ]:
from torch.nn.utils.rnn import pad_sequence

# 각 시퀀스를 텐서로 변환
seqs = [torch.tensor(seq) for seq in seqs]

# 패딩
x = pad_sequence(seqs, batch_first=True)
x

- 각 시퀀스를 텐서로 변환하고, 패딩을 적용하여 배치 처리를 위한 텐서를 준비하는 과정을 실습합니다. 이는 시퀀스 데이터 처리에서 매우 중요한 단계입니다.

## 06-020 텍스트 분류기 모델 정의
- 이 실습에서는 텍스트 분류를 위한 신경망 모델을 정의합니다. TextClassifier 클래스는 입력 데이터를 임베딩 레이어를 통해 고차원 벡터로 변환한 후, LSTM(Long Short-Term Memory) 계층을 사용하여 순차적 특성을 학습합니다. LSTM의 마지막 출력(hidden state)을 이용해 완전 연결 레이어로 클래스를 예측하고, Sigmoid 활성화 함수를 통해 최종 출력을 얻습니다. 이 모델은 자연어 처리 작업에서 시퀀스 데이터를 기반으로 한 분류 작업에 사용됩니다.

In [ ]:
from torch import nn

class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_size, num_class):
        super().__init__()
        # 임베딩 계층 추가
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_class)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.embedding(x)
        out, _ = self.lstm(out)
        out = self.fc(out[:, -1, :])
        out = self.sigmoid(out)
        return out

- 이 실습에서는 LSTM 계층을 포함한 텍스트 분류 모델을 정의하는 과정을 다룹니다. 모델은 임베딩 계층과 LSTM 계층을 활용하여 입력 텍스트의 순차적 특성을 반영한 분류 작업을 수행합니다. LSTM의 최종 출력을 사용하여 여러 클래스로의 분류를 예측합니다.

## 06-021 텍스트 분류기 모델 인스턴스 생성 및 출력 테스트
- 이 실습에서는 TextClassifier 모델의 인스턴스를 생성하고, 입력 데이터에 대한 출력 결과의 형태를 확인합니다. 모델 생성 시, 임베딩 차원과 LSTM의 은닉 상태 크기(hidden_size)를 설정합니다. torch.manual_seed(0)을 사용해 랜덤 시드를 고정하여 재현성을 보장합니다. 모델이 입력 데이터를 기반으로 출력이 올바르게 형성되는지를 테스트합니다.

In [ ]:
torch.manual_seed(0)

hidden_size = 128
# 모델 인스턴스 생성
model = TextClassifier(tokenizer.vocab_size, 2, hidden_size, 1).to(device)

# 출력 테스트
output = model(x[:1].to(device))
output.shape

- 텍스트 분류기 모델을 인스턴스화하고, 입력 데이터를 사용하여 출력의 형태를 확인하는 과정을 실습합니다. 이는 모델이 기대한 대로 작동하는지 검증하는 데 중요한 단계입니다.

## 06-022 장치 적용
- 이 실습에서는 입력 데이터 x와 타겟 데이터 y를 설정한 장치(GPU 또는 CPU)로 이동합니다. PyTorch에서는 모델과 데이터가 동일한 장치에 있어야 연산이 가능하므로, 이를 통해 학습 및 추론을 위한 준비를 합니다.

In [ ]:
# 장치 적용
x = x.to(device)
y = y.to(device)

- 입력 데이터와 타겟 데이터를 장치에 적용하는 과정을 실습합니다. 이는 PyTorch에서 효율적인 계산을 위한 필수적인 단계입니다.

## 06-023 모델 학습 및 손실 계산
- 이 실습에서는 정의한 텍스트 분류기 모델을 학습시키고 손실을 계산하는 과정을 구현합니다. BCELoss를 손실 함수로 사용하고, Adam 옵티마이저를 통해 모델의 파라미터를 업데이트합니다. 지정된 최대 에폭 수만큼 모델을 학습시키며, 주기적으로 손실 값을 출력합니다.

In [ ]:
loss_fn = nn.BCELoss()  # 손실 함수
optimizer = optim.Adam(model.parameters())  # 최적화 도구(optimizer)

epochs = 5000  # 최대 에폭 지정
results = {'cost':[]}

model.train()  # 학습 모드 설정

for epoch in range(epochs):
    h = model(x)  # 예측 값 생성(추론)
    loss = loss_fn(h, y)  # 손실 계산

    optimizer.zero_grad()  # 미분 값 초기화
    loss.backward()  # 역전파(미분 계산)
    optimizer.step()  # 업데이트 진행

    results['cost'].append(loss.item())

    if epoch % 1000 == 0:
        print(f'epoch: {epoch:4d}, cost: {results["cost"][-1]:.10f}')

- 모델을 학습시키고 손실을 계산하여 업데이트하는 과정을 실습합니다. 학습 과정의 손실 변화를 모니터링하는 것은 모델의 성능을 이해하는 데 중요합니다.

## 06-024 예측 값을 클래스로 변환
- 이 실습에서는 모델의 예측 값을 시그모이드 결과를 기반으로 클래스로 변환하는 방법을 학습합니다. h.view(-1)로 예측 결과를 평탄화한 후, 0.5를 기준으로 이진 클래스로 변환합니다. 실제 값과 예측 값을 출력하여 모델의 성능을 평가합니다.

In [ ]:
# 예측 값(시그모이드 값)을 클래스로 변환
pred = (h.view(-1).detach().cpu() > 0.5).to(torch.float32)

print(f'실제 값: {y.view(-1)}')
print(f'예측 값: {pred}')

- 모델의 예측 값을 클래스 형태로 변환하고, 실제 값과 비교하여 출력하는 과정을 실습합니다. 이는 모델의 예측 정확도를 확인하는 데 중요합니다.

---

## 연습문제-06-001 10개 문서 유형 분류를 위한 모델 정의 (Module 방식)
- 이 실습에서는 문서 유형 10개를 분류하기 위한 신경망 모델을 Module 방식으로 정의합니다. 이 모델은 입력된 문서 데이터를 임베딩 레이어와 LSTM 계층을 거쳐 처리한 후, 10개의 출력값을 가지는 완전 연결층을 통해 각 문서가 어느 유형에 속하는지 예측합니다. 활성화 함수는 적용되지 않으며, 출력값을 통해 각 클래스에 대한 로짓 값을 반환합니다.

In [ ]:
import torch
from torch import nn

# DocumentClassifier 모델 정의
class DocumentClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_size, num_class=10):
        super().__init__()

        # 1. Embedding 계층: 입력된 단어를 고차원 벡터로 변환합니다.
        self.embedding = nn.Embedding(vocab_size, embed_dim)  # vocab_size: 단어 사전 크기, embed_dim: 임베딩 차원 수

        # 2. LSTM 계층: 순차적 데이터를 처리하는 LSTM 계층입니다.
        self.lstm = nn.LSTM(embed_dim, hidden_size, batch_first=True)  # embed_dim 크기의 입력을 받아 hidden_size 크기의 출력을 반환

        # 3. Fully Connected Layer (FC): LSTM의 출력을 사용해 문서 유형을 분류하는 계층입니다.
        self.fc = nn.Linear(hidden_size, num_class)  # hidden_size에서 10개의 문서 유형으로 분류

    def forward(self, x):
        # 4. 입력 데이터 임베딩 변환
        out = self.embedding(x)

        # 5. LSTM 계층을 통과시켜 순차적 특성 학습
        out, _ = self.lstm(out)

        # 6. LSTM의 마지막 타임스텝의 출력을 사용해 FC 계층에 전달
        out = self.fc(out[:, -1, :])  # LSTM의 마지막 출력 사용

        # 7. 활성화 함수 없이 10개의 출력값 반환
        return out  # 최종 출력값: (배치 크기, 10개의 문서 유형)

model = DocumentClassifier(vocab_size=10000, embed_dim=128, hidden_size=256)
model(torch.randint(0, 10000, (100, 20))).shape


- 이 실습에서는 PyTorch의 Module 방식을 사용해 문서 유형 10개를 분류하는 모델을 정의합니다. 이 모델은 LSTM을 통해 문서의 순차적 특성을 학습하고, 완전 연결층을 사용하여 10개의 범주로 문서를 분류합니다.

---

# HuggingFace - Transformers

## 06-025 텍스트 분류 파이프라인 사용
- 이 실습에서는 Hugging Face의 Transformers 라이브러리를 사용하여 텍스트 분류 파이프라인을 생성합니다. pipeline() 함수를 통해 'text-classification' 작업을 설정하고, 주어진 문장에 대해 모델의 예측 결과를 확인합니다.

In [ ]:
from transformers import pipeline

classifier = pipeline('text-classification')
classifier("I've been waiting for a HuggingFace course my whole life.")

- 텍스트 분류 파이프라인을 사용하여 입력 문장에 대한 분류 결과를 얻는 과정을 실습합니다. 이는 NLP 작업에서 텍스트의 감정이나 주제를 분류하는 데 유용합니다.

## 06-026 DistilBERT로 텍스트 분류
- 이 실습에서는 Hugging Face의 DistilBERT 모델을 사용하여 텍스트를 분류하는 방법을 보여줍니다. 사전 학습된 DistilBERT 모델을 로드하고, 주어진 문장을 토크나이즈하여 입력 텐서를 생성합니다. 모델에 입력을 전달하여 예측 결과를 얻고, 확률을 계산하여 최종 레이블과 점수를 출력합니다.

In [ ]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

model_name = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
model = DistilBertForSequenceClassification.from_pretrained(model_name)

text = "I've been waiting for a HuggingFace course my whole life."
inputs = tokenizer(text, return_tensors="pt")
with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_id = logits.argmax().item()
label = model.config.id2label[predicted_class_id]
score = torch.softmax(logits, dim=1).squeeze()[predicted_class_id].item()
print(f'label:{label}, score:{score}')

- DistilBERT 모델을 사용하여 입력 텍스트의 감정을 분류하고, 예측 결과와 확률을 출력하는 과정을 실습합니다. 이는 NLP에서 텍스트 분류 작업의 기본적인 방법론입니다.

## 06-027 파이프라인 모델 지정 및 예측
- 이 실습에서는 Hugging Face의 pipeline을 사용하여 이전에 로드한 DistilBERT 모델과 토크나이저를 지정합니다. 지정된 파이프라인을 통해 주어진 문장에 대한 텍스트 분류를 수행하고 결과를 출력합니다.

In [ ]:
# 파이프라인 모델 지정
classifier = pipeline('text-classification', model=model, tokenizer=tokenizer)
classifier("I've been waiting for a HuggingFace course my whole life.")

- 파이프라인을 사용하여 DistilBERT 모델로 텍스트 분류를 수행하고 결과를 출력하는 과정을 실습합니다. 이는 모델을 간편하게 사용하여 예측을 수행하는 방법을 보여줍니다.

## 06-028 여러 문장 분류
- 이 실습에서는 Hugging Face의 파이프라인을 사용하여 여러 개의 문장에 대해 텍스트 분류를 수행합니다. classifier() 함수를 호출하여 입력된 문장 목록을 전달하면, 각 문장에 대한 분류 결과를 얻을 수 있습니다.

In [ ]:
# 여러 문장 분류
classifier(
    ["I've been waiting for a HuggingFace course my whole life.",
     "I hate this so much!"])

- 여러 문장에 대해 텍스트 분류를 수행하고 결과를 출력하는 과정을 실습합니다. 이는 NLP 모델이 동시에 여러 입력을 처리할 수 있는 방법을 보여줍니다.

## 06-029 한국어 감정 분류
- 이 실습에서는 Hugging Face의 파이프라인을 사용하여 한국어 감정 분류 모델을 로드하고, 여러 문장에 대한 감정을 분류합니다. 입력된 문장에 대해 긍정적 또는 부정적인 감정을 예측하여 결과를 출력합니다.

In [ ]:
# 한국어 감정 분류
classifier_ko = pipeline('text-classification', model='matthewburke/korean_sentiment')
classifier_ko(
    ["그녀가 먼저 말을 걸어와서 나는 무척 기뻤다.",
     "듣기 싫어 죽겠네, 짜증나니까 그만 좀 닥쳐줄래"])

- 한국어 감정 분류를 수행하고 결과를 출력하는 과정을 실습합니다. 이는 NLP 모델이 한국어 텍스트의 감정을 분석하는 데 유용한 방법입니다.

## 06-030 제로 샷 분류
- 이 실습에서는 Hugging Face의 제로 샷 분류 파이프라인을 사용하여 주어진 문장이 어떤 클래스에 속하는지를 예측합니다. candidate_labels로 제공된 여러 후보 레이블 중에서 가장 적합한 레이블을 선택하여 결과를 출력합니다.

In [ ]:
# zero-shot classification
classifier = pipeline("zero-shot-classification")
classifier(
    "This is a course about the Transformers library",
    candidate_labels=["education", "politics", "business"],
)

- 제로 샷 분류를 통해 주어진 문장의 분류를 수행하는 과정을 실습합니다. 이는 사전 학습된 모델을 사용하여 새로운 태스크에 대해 적응할 수 있는 방법을 보여줍니다.

## 06-031 한국어 제로 샷 분류
- 이 실습에서는 한국어 문장에 대해 제로 샷 분류를 수행합니다. mDeBERTa 모델을 사용하여 주어진 문장이 어떤 클래스에 속하는지를 예측합니다. 후보 레이블로 제공된 여러 항목 중에서 가장 적합한 레이블을 선택하여 결과를 출력합니다.

In [ ]:
# 한국어 zero-shot classification
model_name = 'MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7'
classifier_ko = pipeline("zero-shot-classification", model_name)
classifier_ko(
    "이 강의는 트랜스포머 라이브러리에 대한 것입니다.",
    candidate_labels=["교육", "정치", "사업"],
)

- 한국어 문장에 대한 제로 샷 분류를 수행하고 결과를 출력하는 과정을 실습합니다. 이는 다국어 모델을 사용하여 다양한 태스크에 적응할 수 있는 방법을 보여줍니다.

## 06-032 한국어 제로 샷 분류 예시
- 이 실습에서는 한국어 문장에 대해 제로 샷 분류를 수행하여 주어진 문장이 어떤 클래스에 속하는지를 예측합니다. 후보 레이블로 제공된 '예술', '정치', '군사' 중에서 가장 적합한 레이블을 선택하여 결과를 출력합니다.

In [ ]:
classifier_ko(
    "이 소총의 유효사거리는 5km를 넘습니다.",
    candidate_labels=["예술", "정치", "군사"],
)

- 주어진 한국어 문장에 대한 제로 샷 분류를 수행하고, 결과를 출력하는 과정을 실습합니다. 이는 NLP 모델이 새로운 태스크에 대해 적응할 수 있는 방법을 보여줍니다.

## 06-033 한국어 제로 샷 분류 예시
- 이 실습에서는 주어진 한국어 문장에 대해 제로 샷 분류를 수행합니다. 문장은 예술 작품에 대한 설명이며, 후보 레이블로 제공된 '예술', '정치', '군사' 중에서 가장 적합한 레이블을 선택하여 결과를 출력합니다.

In [ ]:
classifier_ko(
    "선과 색의 조화가 탁월한 그 작품은 과거의 어느 작가의 그것과 닮아 있다.",
    candidate_labels=["예술", "정치", "군사"],
)

- 주어진 문장에 대한 제로 샷 분류를 수행하고, 출력 결과를 확인하는 과정을 실습합니다. 이는 다양한 주제에 대해 모델의 분류 능력을 평가하는 데 유용합니다.

## 06-034 질문-답변 파이프라인 사용
- 이 실습에서는 Hugging Face의 질문-답변 파이프라인을 사용하여 주어진 질문에 대한 답변을 찾습니다. 질문과 함께 제공된 문맥(context)을 기반으로 모델이 질문에 대한 적절한 답변을 생성합니다.

In [ ]:
# Q & A
question_answerer = pipeline("question-answering")
question_answerer(
    question="Where do I work?",
    context="My name is Sylvain and I work at Hugging Face in Brooklyn",
)

- 질문-답변 파이프라인을 사용하여 주어진 질문에 대한 답변을 찾는 과정을 실습합니다. 이는 NLP 모델이 문맥에서 정보를 추출하는 데 유용한 방법입니다.

## 06-035 한국어 질문-답변 파이프라인 사용
- 이 실습에서는 한국어 질문-답변 파이프라인을 사용하여 주어진 질문에 대한 답변을 찾습니다. 특정 모델을 지정하여 문맥(context)에서 질문에 대한 적절한 답변을 생성합니다.

In [ ]:
# 한국어 Q & A
question_answerer = pipeline("question-answering", model='timpal0l/mdeberta-v3-base-squad2')
question_answerer(
    question="홍길동은 어디에 있지?",
    context="홍길동씨는 잠실에 있는 숨겨진 국가정보원 빌딩에서 청소를 하며 살아 가고있다.",
)

- 한국어 질문-답변 파이프라인을 사용하여 주어진 질문에 대한 답변을 찾는 과정을 실습합니다. 이는 NLP 모델이 한국어 문맥에서 정보를 추출하는 데 유용한 방법입니다.

## 06-036 한국어 질문-답변 추가 예시
- 이 실습에서는 한국어 질문-답변 파이프라인을 사용하여 새로운 질문에 대한 답변을 찾습니다. 주어진 문맥(context)을 바탕으로 질문에 대한 적절한 답변을 생성합니다.

In [ ]:
question_answerer(
    question="홍길동은 무얼하고 있지?",
    context="홍길동씨는 잠실에 있는 숨겨진 국가정보원 빌딩에서 청소를 하며 살아 가고있다.",
)

- 한국어 질문-답변 파이프라인을 사용하여 특정 질문에 대한 답변을 찾는 과정을 실습합니다. 이는 모델이 한국어 문맥에서 정보를 추출하는 능력을 평가하는 데 유용합니다.

## 06-037 GPT 텍스트 생성
- 이 실습에서는 Hugging Face의 텍스트 생성 파이프라인을 사용하여 GPT-2 모델을 기반으로 텍스트를 생성합니다. 주어진 입력 문장에 이어서 모델이 자동으로 텍스트를 생성하도록 합니다.

In [ ]:
# GPT 텍스트 생성
from transformers import pipeline

generator = pipeline("text-generation", model='gpt2')
generator("In this course, we will teach you how to")

- GPT-2 모델을 사용하여 입력 문장에 이어지는 텍스트를 생성하는 과정을 실습합니다. 이는 자연어 생성 모델의 활용 방법을 보여줍니다.

## 06-038 KoGPT 텍스트 생성
- 이 실습에서는 KoGPT 모델을 사용하여 주어진 한국어 문장에 이어지는 텍스트를 생성합니다. '감기에 걸리지 않으려면'이라는 입력 문장에 대해 모델이 생성한 텍스트를 출력하여 확인합니다.

In [ ]:
# KoGPT 텍스트 생성
generator = pipeline("text-generation", model='skt/kogpt2-base-v2')
output = generator("감기에 걸리지 않으려면")
print(output[0]['generated_text'])

- KoGPT 모델을 사용하여 입력 문장에 따른 텍스트를 생성하고, 결과를 출력하는 과정을 실습합니다. 이는 한국어 자연어 생성 모델의 활용을 보여줍니다.